# Molecules - Task 

* Classify the molecules of the validation set using kNN 
* Distance: approximate Graph Edit Distance (GED)


## Data 

We have .gxl files. Those are XMLs with a lot of information. 
→ Only use the chemical symbol node label and the unlabeled, undirected edges:
``` 
    <node id="_1">
        <attr name="symbol">
            <string>C</string>
        </attr>
    </node>
     <edge from="_1" to="_2">
     </edge> 
```


Real example: 
Node 1: 

```
<node id="_1">
    <attr name="symbol">
        <string>O  </string>
    </attr>
    <attr name="chem">
        <int>2</int>
    </attr>
    <attr name="charge">
        <int>0</int>
    </attr><attr name="x">
        <float>4.9921</float>
    </attr>
    <attr name="y">
        <float>1.6561
        </float>
    </attr>
</node>
```

Edges from Node 1: 

```
<edge from="_1" to="_2">
    <attr name="valence"> 
    <int>1</int>
    </attr>
</edge>
```

## Parse Data correctly 


In [97]:
import xml.etree.ElementTree as ET
import os 
import numpy as np

def createLists(filepath):
    tree = ET.parse(filepath)
    root = tree.getroot()
    nodes = []
    edges = []
    symbols = []
    for node in tree.findall('.//node'):
        for key, val in node.attrib.items():
            nodes.append(int(val.replace("_", "")))
    for edge in tree.findall('.//edge'):
        edges.append([int(edge.attrib['from'].replace('_', '')), int(edge.attrib['to'].replace('_', ''))])
    for symbol in tree.findall('.//attr'):
        if(symbol.attrib['name'] == 'symbol'):
            symbols.append([symbol[0].text.replace(' ', '')]) 
    return nodes, edges, symbols 

nodes_ =[]
edges_ = []
molecules = {}
for fn in os.scandir('../Molecules/MoleculesClassification/gxl/'):
    # Only include XML files
    if not fn.name.endswith('.gxl'): 
        continue

    molecules[fn.name.replace('.gxl', '')] = {'symbols': [], 'nodes': [], 'edges' : []}
    molecules[fn.name.replace('.gxl', '')]['symbols'].append(createLists(fn.path)[2])
    molecules[fn.name.replace('.gxl', '')]['nodes'].append(createLists(fn.path)[0])
    molecules[fn.name.replace('.gxl', '')]['edges'].append(createLists(fn.path)[1])


### How to read the dictionary
In molecules all molecules with their nodes and their edges are saved. 

Example: Get all edges from molecule 31510.gxl: 


In [98]:
print(molecules['31510']['edges'])
print(len(molecules['31510']['nodes'][0]))

[[[1, 2], [2, 3], [1, 5], [6, 7], [7, 8], [8, 9], [2, 10], [4, 11], [3, 12], [3, 13], [5, 14], [7, 15], [15, 16], [16, 17], [8, 18], [18, 19], [19, 20], [20, 21], [20, 22], [22, 23], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [29, 30], [30, 31], [31, 32], [32, 33], [33, 34], [34, 35], [4, 5], [9, 10], [6, 10], [4, 13]]]
35


## Task
* Compute approximate GED between pairs of molecules (every molecule from valid set with every molecule from train set) with bipartite graph matching (lecture 10, slide 21)
* Build cost matrix (Dirac)
    * Use Dirac cost function for GED (optimize Cn and Ce) (lecture 9, slide 36)
    * Node substitution: 2*Cn if symbols ≠, 0 otherwise Node deletion/insertion: Cn
    * Edge deletion/insertion: Ce
* Hungarian Algorithm: to find optimal assignment
* Derive edit path costs from the result (distance for classification)

* kNN for classification (optimize for k)
  

### 1. Calculate GED between pairs of molecules, bipartite graph matching 

### Bipartite graph matching procedure: 
![BP_image](BP_image.jpg)

#### Cost Matrix: 
1.  Upper left: substitutions (nodes plus adjacent edges) 
2.  Upper right: deletions (nodes plus adjacent edges) 
3.  Lower left: insertions (nodes plus adjacent edges)
4.  Lower right: dummy assignments (ε → ε)

In [109]:
def BP(graph1, graph2, Cn, Ce): 
    # build cost matrix according to the input graphs g1 and g2 
    n = len(molecules[graph1]['nodes'][0])
    m = len(molecules[graph2]['nodes'][0])
    l = n + m
    matrix = np.zeros((l, l))
    
    ##upper left: substitutions 
    for i in range(0, n):
        for j in range(0, m):
            if (molecules[graph1]['symbols'][0][i] == molecules[graph2]['symbols'][0][j]):
                #print(molecules[graph1]['symbols'][0][i], molecules[graph2]['symbols'][0][j]) 
                matrix.itemset((i, j), 1)
    return matrix

BP('31510', '10071', 1, 1)
# build cost matrix according to the input graphs g1 and g2 


array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [103]:
# Zusatzmethod 
# evt statt a und i 0 und i verwenden 
def label(label):
    label  = label.strip()
    if label == 'i':
        return 0;
    if label == 'a':
        return 1;
    return -1;

array = ['a', 'i', 'a','a', 'i']
striped_array = []
for i in array:
    striped_array.append(label(i))
print(striped_array) 

[1, 0, 1, 1, 0]
